***
## Предварительные настройки

По умолчанию настройки админ-зоны описаны в классе `ModelAdmin` приложения `admin`. Чтобы изменить интерфейс админки — следует переопределить дефолтные настройки этого класса. Для этого нужно:

1. Создать собственный класс, унаследовав его от `admin.ModelAdmin`

2. Зарегистрировать этот новый класс как источник настроек админ-зоны

3. В унаследованном классе переопределить настройки интерфейса админки

Админ-зона настраивается отдельно для каждой модели. Настроим интерфейс для модели `IceCream`.

```py
# ice_cream/admin.py
...

# Создаём класс, в котором будем описывать настройки админки:
class IceCreamAdmin(admin.ModelAdmin):
    # В этом классе опишем все настройки, какие захотим.
    ...

# Регистрируем новый класс: 
# указываем, что для отображения админки модели IceCream
# вместо стандартного класса нужно использовать класс IceCreamAdmin 
admin.site.register(IceCream, IceCreamAdmin)

...
```

***
## Всё нужное на одной странице

Начнём с отображения полей в разделе «Мороженое»: в списке сортов мороженого для каждой записи выведем все её поля, а не только название.

В классе `IceCreamAdmin` можно указать:

* какие поля будут показаны на странице списка объектов (свойство `list_display`, это кортеж);

* какие поля можно редактировать прямо на странице списка объектов (свойство `list_editable`, кортеж);

* `search_fields` — кортеж с перечнем полей, по которым будет проводиться поиск. Форма поиска отображается над списком элементов.

* `list_filter` — кортеж с полями, по которым можно фильтровать записи. Фильтры отобразятся справа от списка элементов.

* В кортеже `list_display_links` указывают поля, при клике на которые можно перейти на страницу просмотра и редактирования записи. По умолчанию такой ссылкой служит первое отображаемое поле.

```py
# ice_cream/admin.py
...

class IceCreamAdmin(admin.ModelAdmin):
    list_display = (
        'title',
        'description',
        'is_published',
        'is_on_main',
        'category',
        'wrapper'
    )
    list_editable = (
        'is_published',
        'is_on_main',
        'category'
    )    
    search_fields = ('title',) 
    list_filter = ('category',)
    list_display_links = ('title',)

# Регистрируем кастомное представление админ-зоны
admin.site.register(IceCream, IceCreamAdmin)

...
```

Админка стала удобнее: теперь в списке объектов видно не только название сорта мороженого, как было при дефолтных настройках, но и содержимое полей, перечисленных в кортеже `list_display`.

![alt text](https://pictures.s3.yandex.net/resources/S3.2_29_1679043597.png)

В полях типа `ForeignKey` будет выведено значение, определённое в методе `__str__` связанной модели; например, в поле «Категория» показывается значение поля `title` связанной записи из модели `Category`, ведь в этой модели при помощи метода `__str__` установлено значение `self.title`. Если бы в модели `Category` метод `__str__` не был переопределён — в поле «Категория» было бы показано что-то вроде `Category object(2)`.

***
## Пустое поле

Для каждого сорта мороженого в колонке «Обёртка» выводится содержимое поля `wrapper`. Это поле необязательно для заполнения; в тех записях, где это поле не определено, в колонке «Обёртка» отображается прочерк, символ `-`. 

Выглядит не очень хорошо: вместо дефиса лучше бы выводить какое-нибудь пояснение. Для этого нужно переопределить свойство `empty_value_display` в модели `IceCreamAdmin`:

```py
# ice_cream/admin.py
...

class IceCreamAdmin(admin.ModelAdmin):
    ...
    # Это свойство сработает для всех полей этой модели.
    # Вместо пустого значения будет выводиться строка "Не задано".
    empty_value_display = 'Не задано'
... 
```

Переопределить отображение пустых полей можно и на уровне приложения — тогда во всех моделях приложения пустые поля будут отображаться одинаково:

```py
# ice_cream/admin.py
from django.contrib import admin

from .models import Category, IceCream, Topping, Wrapper

...
# Этот вариант сработает для всех моделей приложения.
# Вместо пустого значения в админке будет отображена строка "Не задано".
admin.site.empty_value_display = 'Не задано' 
```

***
## Интерфейс для управления записями, связанными «многие ко многим»

Модели `IceCream` и `Topping` связаны как «многие ко многим». Чтобы привязать к мороженому один или несколько топпингов — штатная админка Django предлагает выбрать нужные топпинги из списка, кликая по ним при нажатой кнопке Ctrl (на Windows) или Command (на macOS). Для небольшого списка топпингов такой интерфейс хорош, но если в базе их больше десятка — администратор намучается.

Убережём администратора от напрасных мучений: изменим интерфейс так, чтобы связанные записи можно было перекладывать из одного окошка в другое. Для подключения этого интерфейса в классе с настройками нужно переопределить свойство `filter_horizontal`:

```py
# ice_cream/admin.py
...

class IceCreamAdmin(admin.ModelAdmin):
    ...
    # Указываем, для каких связанных моделей нужно включить такой интерфейс:
    filter_horizontal = ('toppings',)
... 
```

***
## Все связанные записи на одной странице

Интерфейс администратора можно настроить так, чтобы на странице редактирования определённой записи отображались связанные с ней записи другой модели. Например, на страницу редактирования категории можно подгрузить блок с информацией о связанных с ней сортах мороженого. 

Такие блоки называют «вставки», для их настройки в Django есть классы `admin.TabularInline` и `admin.StackedInline`. Разница между этими классами заключается лишь в способе отображения связанных записей: `TabularInline` отображает поля вставки в строку, а `StackedInline` — столбцом, одно под другим.

Подключим вставку сортов мороженого на страницу категории:

```py
# ice_cream/admin.py
...

# Подготавливаем модель IceCream для вставки на страницу другой модели.
class IceCreamInline(admin.StackedInline):
    model = IceCream
    extra = 0

class CategoryAdmin(admin.ModelAdmin):
    inlines = (
        IceCreamInline,
    )
    list_display = (
        'title',        
    )

admin.site.register(Category, CategoryAdmin) 
```

Вуаля: управлять сортами мороженого, связанными с выбранной категорией, можно прямо на странице категории. Связанные с категорией сорта можно отредактировать или удалить; можно привязать к категории другие сорта мороженого.

***
## Подсказки для полей модели

Добавим финальный штрих — под названиями полей выведем подсказки, описывающие, какое содержимое ожидается в этом поле. Подсказка указывается в поле модели, в именованном аргументе `help_text`:

```py
# ice_cream/models.py
class Wrapper(PublishedModel):
    title = models.CharField(
        'Название',
        max_length=256,
        help_text='Уникальное название обёртки, не более 256 символов'
    )

    class Meta:        
        verbose_name = 'обёртка' 
``` 

![alt text](https://pictures.s3.yandex.net/resources/S3.2_33-1_1679043713.png)


К сожалению, Django понятия не имеет о том, что в русском языке есть склонения, и в результате заголовки не всегда получаются правильными. Можно выдумать какой-нибудь хитрый оборот и задать, например, значение

```py
...
verbose_name = 'объект «Обёртка»'
... 
```

***
## Хозяйке на заметку

* Отображение списка объектов на странице модели настраивается довольно гибко: например, можно управлять порядком отображения или количеством элементов на экране. Все эти параметры описаны [в специальном разделе документации](https://docs.djangoproject.com/en/3.2/ref/contrib/admin/#django.contrib.admin.ModelAdmin).

* Выводить связанные модели в общем интерфейсе админки можно несколькими способами; описание — [на этой странице документации](https://docs.djangoproject.com/en/3.2/ref/contrib/admin/#inlinemodeladmin-objects).

* Поваренная книга разработчика: помимо [официальной документации](https://docs.djangoproject.com/en/3.2/ref/contrib/admin/), бывает полезен сборник рецептов «приготовления» Django-админки: [Django Admin Cookbook](https://books.agiliq.com/projects/django-admin-cookbook/en/latest/).

Рецепты ориентированы на Django 2, но большинство рекомендаций применимы и для новых версий фреймворка.

> При работе с админ-зоной используйте [шпаргалку](https://code.s3.yandex.net/Python-dev/cheatsheets/030-django-admin-zona-shpora/030-django-admin-zona-shpora.html). В ней собраны основные команды, а также есть базовая информация о настройках.